# I  have not use PostgreSQL for this task ,i have knowledge of  mysql, snowflake, sql server, bigquery for this task

# so i don't  have knowledge how to do in PostgreSQL, BUT I AM READY TO LEARN 



## NOW I AM DOING BY USING QUERY, AS WELL AS BY READING CSV FILE
# FIRSTLY I AM CREATING PIPELING USING QUERY 


## I HAVE USE JUST 1 QUERY BUT YOU CAN MULTIPLE QUERY LET SAY 10 OR 20 OR MORE QUERY IN THAT LIST

In [ ]:
import os
os.sys.path.append('/opt/airflow/dags/')
from helper_func.connections import data_fetch
import pandas as pd
import pymysql
import pendulum
from datetime import timedelta,datetime, date
import numpy as np
import pandas as pd
from pretty_html_table import build_table
from prettytable import PrettyTable

In [ ]:
CondList = {
# Sales  Summary
'Revenue' : {
     'cloths sales ' : {'cloths_type': ["boy cloth","girls cloth"], 'sales':'sum'}, 
      'electronic sales' : {'electronic_type': ["mobile","laptop"], 'sales':'sum'},
      'books sale' : {'book_type': ["book","novel"], 'sales':'sum'}},}

##  you can write various query let 10 or 20 query and join   and  isin,like,group by....etc as per requirements in this list i.e Qlist


In [ ]:
def queries(start_epoch, end_epoch):
    Qlist = [f'''select txn_date as n_time, cloths_type,electronic_type,book_type ,sales_amt as sales from sales_txn_history 
    where txn_date >= {start_epoch} and txn_date < {end_epoch}''']      

    return Qlist

In [ ]:
def mis(st):
#     date in format = yyyy,mm,dd
    '''
    st is datetime
    
    '''
#     st = datetime(yyyy,mm,dd)
    c_date = st.date() - timedelta(days=1)
    start_date = datetime(c_date.year,c_date.month,1)
    start_epoch = int(start_date.strftime('%s')) - 19800   # datetimeindex is in UTC so to convert into indian time i uses 19,800 sec i.e 5.5hr
    end_epoch = int(st.date().strftime('%s')) - 19800
    start_dpoch = int(c_date.strftime('%s')) - 19800
    end_dpoch = int(st.date().strftime('%s')) - 19800
    
    print(start_epoch)
    print(end_epoch)
    
#     Connection with source
    fetch_class = data_fetch()
    fetch_class.set_mysql_prod()
    
#  initialisation
    index1 = 0
    data = []
    temp_list = []
    
#  fetching queries
    Qlist  = queries(start_epoch, end_epoch)
    
    for id, info in CondList.items():
        print("\n ID:", id, index1)
        df_temp = pd.DataFrame()
    
        try:
            data= fetch_class.mysql_query(Qlist[index1])
        
        except Exception as e:
            print("error1",e)
            index1 =index1 + 1

            continue
        
        
        
        
        
        for key in info:
            field, k = info[key].popitem()
                try:
                    val=data.loc[(data[list(info[key])].isin(info[key])).all(axis=1),field].agg(func = k)
                    val2=data[(data['n_time'] >= start_dpoch) & (data['n_time'] < end_dpoch)].loc[(data[list(info[key])].isin(info[key])).all(axis=1),field].agg(func = k)

                except Exception as e:
                    print("error2",e)

                    try:
                        val=data[field].agg(func = k)
                        val2=data[(data['n_time'] >= start_dpoch) & (data['n_time'] < end_dpoch)].loc[(data[list(info[key])]).all(axis=1),field].agg(func = k)

                    except  Exception as e:
                        print("error3",e)
                        val=0
                        val2=0
        
# val is giving data of   Last Day sales i.e today()-1
# val2 is giving sales data of MTD 
                
            temp_list.append([id,key, int(round(val2,0)), int(round(val,0))])
        
        index1 = index1 + 1
    
    
    df = pd.DataFrame(temp_list, columns = ['Category','Field','Last Day Data','MTD'])
    df = df.transpose()
    df.drop(index = 'Category', inplace = True)
    df.reset_index(inplace = True)
    new_header = df.iloc[0]
    df = df[1:]
    df.columns = new_header
    
    df.drop(columns = ['Overall sales','salesback','Bonus Issued','Total Credit','Total Debit','Profit-Loss'], inplace = True)
    print(df.columns)
    
    df = df[['Field','cloths_type','electronic_type','book_type','sales']]
    df_final = df.transpose()
    df_final.reset_index(inplace = True)
    new_header = df_final.iloc[0]
    df_final = df_final[1:]
    df_final.columns = new_header
    
    # data cleaning
    df_final.dropna(inplace=True)
    df_final.drop_duplicates(inplace=True)
    
    output = build_table(df_final ,'blue_light')
    
    
    count=0
    ############# to find the exact match and replacing it style which gives padding to the cells
    output=output.replace('<td style = "background-color: #D9E1F2;font-family: Century Gothic, sans-serif;font-size: medium;text-align: left;padding: 0px 20px 0px 0px;width: auto">','<td style = "background-color: white;font-family: Century Gothic, sans-serif;font-size: medium;text-align: left;width: auto;border: 1px solid black;padding: 5px; text-align: left; ">')
    output=output.replace('<td style = "background-color: #FFFFFF;font-family: Century Gothic, sans-serif;font-size: medium;text-align: left;padding: 0px 20px 0px 0px;width: auto">','<td style = "background-color:white;font-family: Century Gothic, sans-serif;font-size: medium;text-align: left;width: auto;border: 1px solid black;padding: 5px; text-align: left; ">')
    output=output.replace('<td style = "background-color: white;font-family: Century Gothic, sans-serif;font-size: medium;text-align: left;padding: 0px 20px 0px 0px;width: auto">','<td style = "background-color: white;font-family: Century Gothic, sans-serif;font-size: medium;text-align: left;width: auto;border: 1px solid black;padding: 5px; text-align: left; ">')
    output=output.replace('<td style = "background-color: white; color: black;font-family: Century Gothic, sans-serif;font-size: medium;text-align: left;padding: 0px 20px 0px 0px;width: auto">','<td style = "background-color: white;font-family: Century Gothic, sans-serif;font-size: medium;text-align: left;width: auto;border: 1px solid black;padding: 5px; text-align: left; ">')
    output=output.replace('<th style = "background-color: #D9E1F2;font-family: Century Gothic, sans-serif;font-size: medium;text-align: left;width: auto;border: 1px solid black;padding: 5px; text-align: left; ">','<th>')

    #############looping to find the exact match and replacing it with the header tag
    for change in toBeChangedRow:
        output=output.replace('<td style = "background-color: white;font-family: Century Gothic, sans-serif;font-size: medium;text-align: left;width: auto;border: 1px solid black;padding: 5px; text-align: left; ">{}</td>\n'.format(change),'</tr><tr><td style = "background-color: #FFFFFF;font-family: Century Gothic, sans-serif;font-size: medium;text-align: left;width: auto;border: 1px solid black;padding: 5px; text-align: left; "><strong>{}</strong></td>\n<td style = "background-color: #FFFFFF;font-family: Century Gothic, sans-serif;font-size: medium;text-align: left;width: auto;border: 1px solid black;padding: 5px; text-align: left; "></td>\n<td style = "background-color: #FFFFFF;font-family: Century Gothic, sans-serif;font-size: medium;text-align: left;width: auto;border: 1px solid black;padding: 5px; text-align: left; "></td>\n</tr><tr><td style = "background-color: white;font-family: Century Gothic, sans-serif;font-size: medium;text-align: left;width: auto;border: 1px solid black;padding: 5px; text-align: left; ">{}</td>\n'.format(Headings[count],change))
        count=count+1
    for i,v in df_final['Last Day Data'].items():
        if v>0:
            output=output.replace('<td style = "background-color: white;font-family: Century Gothic, sans-serif;font-size: medium;text-align: left;width: auto;border: 1px solid black;padding: 5px; text-align: left; ">{}</td>'.format(v),'<td style = "background-color: white;font-family: Century Gothic, sans-serif;font-size: medium;text-align: left;width: auto;border: 1px solid black;padding: 5px; text-align: left; ">{}</td>'.format("{:,}".format(v)))

    for i,v in df_final['MTD'].items():
        if v>0:
            output=output.replace('<td style = "background-color: white;font-family: Century Gothic, sans-serif;font-size: medium;text-align: left;width: auto;border: 1px solid black;padding: 5px; text-align: left; ">{}</td>'.format(v),'<td style = "background-color: white;font-family: Century Gothic, sans-serif;font-size: medium;text-align: left;width: auto;border: 1px solid black;padding: 5px; text-align: left; ">{}</td>'.format("{:,}".format(v)))
    
    output=output.replace('<th style = "background-color: #FFFFFF;font-family: Century Gothic, sans-serif;font-size: medium;color: #305496;text-align: left;border-bottom: 2px solid #305496;padding: 0px 20px 0px 0px;width: auto"></th>\n','')
    
    return(output)

## it will create 4 columns 'Category','Field','Last Day Data','MTD'
## these 4 column have two types of data
## 1. last day sales data
## 2. MTD will return sales of  first date of sales date till today-1 of this month

In [ ]:
default_args = {
    'owner': 'BHOLU SINGH',
    'depends_on_past': False,
    'start_date': datetime(2023, 5, 13),
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
    
    'email': ['bholusing.data@gmail.com'],  # Specify the email ID of the DAG creator
    'email_on_failure': True,
    'email_on_retry': True,
    'email_to': ['bholusing.data@gmail.com', 'anyname@gmail.com','anyname2@gmail.com'],}

dag = DAG(
    'sales_pipeline',
    default_args=default_args,
    description='ETL pipeline for processing sales data',
    schedule_interval='0 0 * * *',
    #schedule_interval='@daily',
    catchup=False,)




In [ ]:
def extract_data():
    my_dataframe = mis(st)
    return my_dataframe

def load_data():
    transformed_data = my_dataframe()
    return my_dataframe

extract_task = PythonOperator(task_id='extract_task',python_callable=extract_data,dag=dag,)
load_task = PythonOperator(task_id='load_task',python_callable=load_data,dag=dag,)

extract_task >> transform_task >> load_task

# I CAN ALSO CREATE A METHOD IN WILL YOU CAN STORE ALL THE ABOVE LOGIC

# Now use VS CODE IDE and run this above code and go to local host of  AIRFLOW and trigger DAG 


# It will send reports/summary on given mail on daily basis on scheduled time 

#### ------------------------------------------------------------------------------------------------------------------------------------------------------
# VIA CSV FILE READING

In [ ]:
from datetime import datetime
import pandas as pd
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from airflow.operators.email_operator import EmailOperator

# Define the DAG
default_args = {'owner': 'BHOLU SINGH','start_date': datetime(2023, 5, 13),}

dag = DAG('sales_pipeline',default_args=default_args,schedule_interval='0 0 * * *',)

# Define the tasks
def extract_data():
    df = pd.read_csv('/path/to/your/sales_data.csv')
    return df

def transform_data(df):
    df.drop_duplicates(inplace=True)
    df.dropna(inplace=True)
    return df

def load_data(df):
    df.to_csv('/path/to/save/transformed_data.csv', index=False)
    return html

def send_email():
    # Send an email
    subject = 'sales Pipeline Report'
    body = 'The sales pipeline has completed successfully.'
    to = 'bholu.data@gmail.com'
    email_task = EmailOperator(
        task_id='send_email_task',
        to=to,
        subject=subject,
        html_content=body,
        dag=dag,
    )
    return email_task

# Define the tasks using PythonOperator
extract_task = PythonOperator(
    task_id='extract_task',
    python_callable=extract_data,
    dag=dag,
)

transform_task = PythonOperator(
    task_id='transform_task',
    python_callable=transform_data,
    dag=dag,
)

load_task = PythonOperator(
    task_id='load_task',
    python_callable=load_data,
    dag=dag,
)

email_task = PythonOperator(
    task_id='send_email_task',
    python_callable=send_email,
    dag=dag,
)

# Set task dependencies
extract_task >> transform_task >> load_task >> email_task


# in above code you can use various concept also like select only required column,transform specific column as per requirements....etc